<h1> aWhere API single calls from a list of lat-lons </h1>

In [5]:
import datetime
import pandas as pd

from __future__ import division
from awhere import Awhere

DATE_FORMAT = '%Y-%m-%d'

# Setup key and sectret in aWhere.py
awhere = Awhere()

<h1> import information from example woreda lat-lon file </h1>

In [5]:
#region_df = pd.read_csv('/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv')
woreda_df = pd.read_csv('lat_lons/woreda_info.csv')

In [ ]:
START_DATE = '2013-01-01'
LAST_DATE = (datetime.datetime.now() - datetime.timedelta(days=1))
LAST_DATE_STR = LAST_DATE.strftime(DATE_FORMAT)

for lat, lon, geoKey in woreda_df[['WoredaLat', 'WoredaLon', 'GeoKey']].values:
    # Check it lat/lon is valid
    if lat !=0:
        # Reset the start date for every Woreda
        start_date = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
        start_date_str = start_date.strftime(DATE_FORMAT)
        results_df = []

        while start_date < LAST_DATE : 
            result = awhere.single_call(lat, lon, start_date_str, LAST_DATE_STR)
            result_df = pd.DataFrame(awhere.flatten_single(result))
            start_date = datetime.datetime.strptime(result_df.iloc[-1]['date'], DATE_FORMAT) + datetime.timedelta(days=1)
            start_date_str = start_date.strftime(DATE_FORMAT)
            results_df.append(result_df)

        dataframe = pd.concat(results_df, ignore_index=True)
        dataframe['GeoKey']  = geoKey
        dataframe.to_csv('data/%s.csv' % geoKey.replace(' ','_'))